# Imports

In [399]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from pandas import DataFrame
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [400]:
data = pd.read_excel('data.xlsx')
train, test = train_test_split(data, test_size=0.2)
train_copy = train
test_copy = test

# Features

# One-Hot Encoding

In [401]:
def onehot(data):
    data = data.rename(columns = {'In Comp':'comp'})
    gender = {'M': 0,'F': 1}
    incomp = {'Yes': 1,'No': 0}
    data.Gender = [gender[item] for item in data.Gender]
    data.comp = [incomp[item] for item in data.comp]
    return data
train = onehot(data)
clusters = len(train.ID_random.unique())

In [402]:
train.head()

,ID_random,Gender,SpecificGravity,comp,Adiol,Bdiol,Androsterone,Etiocholanolone,Epitestosterone,Testosterone,...,Andro_Etio_ratio,Adiol_Bdiol_ratio,Adiol_E_ratio,Adiol_corr,Bdiol_corr,Androsterone_corr,Etiocholanolone_corr,Epitestosterone_corr,Testosterone_corr,Total Observation
0,1,0,1.005,1,6.76,4.70,381.56,165.13,4.04,3.51,...,2.31,1.44,1.67,27.04,18.79,1526.22,660.53,16.18,14.02,4
1,1,0,1.006,1,10.21,5.57,807.43,270.68,5.90,1.84,...,2.98,1.83,1.73,34.03,18.58,2691.43,902.25,19.68,6.15,4
2,1,0,1.008,1,9.97,6.54,732.11,330.76,5.41,0.94,...,2.21,1.52,1.84,24.92,16.35,1830.27,826.90,13.52,2.34,4
3,1,0,1.004,1,10.64,5.05,542.51,192.36,4.08,1.45,...,2.82,2.10,2.61,53.18,25.27,2712.53,961.79,20.40,7.26,4
4,6,0,1.008,0,29.90,69.90,1526.00,672.00,17.40,20.40,...,2.27,0.43,1.72,74.75,174.75,3815.00,1680.00,43.50,51.00,6


# Naive Clustering

In [403]:
km1 = KMeans(
    n_clusters=clusters, init='random',
    n_init=10, max_iter=800, 
    tol=1e-04, random_state=0
)

In [404]:
y_km = km1.fit(train)

In [405]:
from sklearn import metrics
labels = y_km.labels_
metrics.silhouette_score(train, labels, metric = 'euclidean')

0.14177152805187254

In [406]:
#centroids = km.cluster_centers_
#centroids[0]

# Some Feature Engineering

In [407]:
def min_max_scaling(column):
    return (column-column.min()) / (column.max()-column.min())

# Normalizing

In [408]:
train['SpecificGravity'] = min_max_scaling(data['SpecificGravity'])

Pearson Correlation

In [409]:
#plt.matshow(data.corr())
#plt.show()
corr_mat = train.corr(method='pearson')
corr_mat.style.background_gradient(cmap='coolwarm')
# 'RdBu_r', 'BrBG_r', & PuOr_r are other good diverging colormaps

,ID_random,Gender,SpecificGravity,comp,Adiol,Bdiol,Androsterone,Etiocholanolone,Epitestosterone,Testosterone,T_E_ratio,Andro_T_ratio,Andro_Etio_ratio,Adiol_Bdiol_ratio,Adiol_E_ratio,Adiol_corr,Bdiol_corr,Androsterone_corr,Etiocholanolone_corr,Epitestosterone_corr,Testosterone_corr,Total Observation
ID_random,1.000000,-0.007288,-0.020737,-0.013133,-0.023723,-0.024515,-0.012391,-0.009202,-0.015809,-0.008651,0.002526,-0.007834,-0.018403,-0.004989,-0.010035,-0.010552,-0.006641,0.005015,0.016502,-0.005876,0.005225,-0.014664
Gender,-0.007288,1.000000,-0.061799,-0.046676,-0.317988,-0.208454,-0.148932,0.023537,-0.362998,-0.391713,-0.144558,0.236933,-0.203264,-0.156338,0.136448,-0.340866,-0.223882,-0.146070,0.095564,-0.412352,-0.448750,0.033658
SpecificGravity,-0.020737,-0.061799,1.000000,-0.164034,0.468104,0.416042,0.565028,0.588333,0.441644,0.396179,-0.014537,0.064145,-0.029100,0.007527,-0.014899,0.041840,0.048686,-0.013714,0.008851,0.048219,0.036165,0.079152
comp,-0.013133,-0.046676,-0.164034,1.000000,-0.045050,-0.109020,0.032873,-0.106401,-0.110582,-0.052681,0.084602,-0.009829,0.164228,0.125550,0.037970,0.029119,-0.062894,0.176153,-0.007606,-0.042960,0.019000,-0.136650
Adiol,-0.023723,-0.317988,0.468104,-0.045050,1.000000,0.649238,0.726264,0.490680,0.601599,0.637871,0.084210,-0.086994,0.221807,0.147737,0.231754,0.816893,0.500299,0.500769,0.252474,0.425808,0.474636,-0.047770
Bdiol,-0.024515,-0.208454,0.416042,-0.109020,0.649238,1.000000,0.522486,0.640773,0.466301,0.664961,0.202811,-0.208487,-0.096756,-0.357721,0.084193,0.471003,0.862957,0.305163,0.448073,0.304734,0.518094,0.020327
Androsterone,-0.012391,-0.148932,0.565028,0.032873,0.726264,0.522486,1.000000,0.711852,0.569957,0.538552,0.005912,0.086842,0.270815,0.140232,0.063569,0.486145,0.325891,0.750373,0.436782,0.363183,0.346622,-0.037946
Etiocholanolone,-0.009202,0.023537,0.588333,-0.106401,0.490680,0.640773,0.711852,1.000000,0.452812,0.459307,0.016720,0.041782,-0.240340,-0.217363,0.005528,0.243721,0.441556,0.409622,0.748338,0.224974,0.251420,0.067667
Epitestosterone,-0.015809,-0.362998,0.441644,-0.110582,0.601599,0.466301,0.569957,0.452812,1.000000,0.591126,-0.218631,-0.097233,0.108618,0.062415,-0.283818,0.410150,0.312771,0.351163,0.220159,0.853446,0.449122,0.032940
Testosterone,-0.008651,-0.391713,0.396179,-0.052681,0.637871,0.664961,0.538552,0.459307,0.591126,1.000000,0.387911,-0.320457,0.074496,-0.202538,-0.049300,0.458332,0.530436,0.339558,0.255144,0.449699,0.865322,0.013454


Adiol-Androsterone
Adiol-Adiol_corr
Bdiol-Bdiol_corr
Epitestosterone-Epitestosterone_corr
Testosterone-Testosterone_corr
Androsterone-Etiocholanolone

Based on the above observations, we drop:
1. Adiol
2. Bdiol
3. Epitestosterone
4. Etiocholanolone
5. Testosterone
6. Androsterone
7. Total Observation
8. ID_random

Note - We do not drop Testosterone and Testosterone_corr as we are aware of the preservation of both the magnitudes from the domain literature.

Dropping

In [410]:
def drop(data_dropped):
    data_dropped = data_dropped.drop(['ID_random'], axis=1)
    data_dropped = data_dropped.drop(['Adiol'], axis=1)
    data_dropped = data_dropped.drop(['Bdiol'], axis=1)
    data_dropped = data_dropped.drop(['Epitestosterone_corr'], axis=1)
    data_dropped = data_dropped.drop(['Etiocholanolone_corr'], axis=1)
    data_dropped = data_dropped.drop(['Androsterone_corr'], axis=1)
    data_dropped = data_dropped.drop(['Testosterone'], axis=1)
    data_dropped = data_dropped.drop(['Etiocholanolone'], axis=1)
    data_dropped = data_dropped.drop(['Total Observation'], axis=1)
    #data_dropped = data_dropped.drop(['Androsterone'], axis=1)
    #data_dropped = data_dropped.drop(['Testosterone'], axis=1)
    return data_dropped



data_dropped = drop(train)

In [411]:
corr_mat = data_dropped.corr(method='pearson')
corr_mat.style.background_gradient(cmap='coolwarm')

,Gender,SpecificGravity,comp,Androsterone,Epitestosterone,T_E_ratio,Andro_T_ratio,Andro_Etio_ratio,Adiol_Bdiol_ratio,Adiol_E_ratio,Adiol_corr,Bdiol_corr,Testosterone_corr
Gender,1.000000,-0.061799,-0.046676,-0.148932,-0.362998,-0.144558,0.236933,-0.203264,-0.156338,0.136448,-0.340866,-0.223882,-0.448750
SpecificGravity,-0.061799,1.000000,-0.164034,0.565028,0.441644,-0.014537,0.064145,-0.029100,0.007527,-0.014899,0.041840,0.048686,0.036165
comp,-0.046676,-0.164034,1.000000,0.032873,-0.110582,0.084602,-0.009829,0.164228,0.125550,0.037970,0.029119,-0.062894,0.019000
Androsterone,-0.148932,0.565028,0.032873,1.000000,0.569957,0.005912,0.086842,0.270815,0.140232,0.063569,0.486145,0.325891,0.346622
Epitestosterone,-0.362998,0.441644,-0.110582,0.569957,1.000000,-0.218631,-0.097233,0.108618,0.062415,-0.283818,0.410150,0.312771,0.449122
T_E_ratio,-0.144558,-0.014537,0.084602,0.005912,-0.218631,1.000000,-0.328321,-0.012930,-0.289299,0.368376,0.094184,0.242729,0.455878
Andro_T_ratio,0.236933,0.064145,-0.009829,0.086842,-0.097233,-0.328321,1.000000,0.053934,0.395555,0.045141,-0.111868,-0.253492,-0.378972
Andro_Etio_ratio,-0.203264,-0.029100,0.164228,0.270815,0.108618,-0.012930,0.053934,1.000000,0.474745,0.067844,0.263511,-0.108997,0.107752
Adiol_Bdiol_ratio,-0.156338,0.007527,0.125550,0.140232,0.062415,-0.289299,0.395555,0.474745,1.000000,0.080058,0.175899,-0.428172,-0.228684
Adiol_E_ratio,0.136448,-0.014899,0.037970,0.063569,-0.283818,0.368376,0.045141,0.067844,0.080058,1.000000,0.399001,0.139624,-0.061190


# Updated Clusters

In [412]:
km_up = KMeans(
    n_clusters=clusters, init='random',
    n_init=50, max_iter=4000, 
    tol=1e-06, random_state=0
)

In [413]:
y_km_up = km_up.fit(data_dropped)
labels_up = y_km_up.labels_
metrics.silhouette_score(data_dropped, labels_up, metric = 'euclidean')

0.15497658244069662

Assigning Clusters to each Sample

In [414]:
data_dropped['clusters'] = km_up.predict(data_dropped)
data_dropped.head()

,Gender,SpecificGravity,comp,Androsterone,Epitestosterone,T_E_ratio,Andro_T_ratio,Andro_Etio_ratio,Adiol_Bdiol_ratio,Adiol_E_ratio,Adiol_corr,Bdiol_corr,Testosterone_corr,clusters
0,0,0.095238,1,381.56,4.04,0.87,108.83,2.31,1.44,1.67,27.04,18.79,14.02,355
1,0,0.119048,1,807.43,5.90,0.31,437.90,2.98,1.83,1.73,34.03,18.58,6.15,1166
2,0,0.166667,1,732.11,5.41,0.17,782.16,2.21,1.52,1.84,24.92,16.35,2.34,2018
3,0,0.071429,1,542.51,4.08,0.36,373.52,2.82,2.10,2.61,53.18,25.27,7.26,2690
4,0,0.166667,0,1526.00,17.40,1.17,74.80,2.27,0.43,1.72,74.75,174.75,51.00,1621


Adding ID back to the dataframe to extract IDs cluster wise

In [415]:
ID_col = data['ID_random']
data_dropped['ID_added'] = ID_col
data_dropped.head()

,Gender,SpecificGravity,comp,Androsterone,Epitestosterone,T_E_ratio,Andro_T_ratio,Andro_Etio_ratio,Adiol_Bdiol_ratio,Adiol_E_ratio,Adiol_corr,Bdiol_corr,Testosterone_corr,clusters,ID_added
0,0,0.095238,1,381.56,4.04,0.87,108.83,2.31,1.44,1.67,27.04,18.79,14.02,355,1
1,0,0.119048,1,807.43,5.90,0.31,437.90,2.98,1.83,1.73,34.03,18.58,6.15,1166,1
2,0,0.166667,1,732.11,5.41,0.17,782.16,2.21,1.52,1.84,24.92,16.35,2.34,2018,1
3,0,0.071429,1,542.51,4.08,0.36,373.52,2.82,2.10,2.61,53.18,25.27,7.26,2690,1
4,0,0.166667,0,1526.00,17.40,1.17,74.80,2.27,0.43,1.72,74.75,174.75,51.00,1621,6


In [416]:
mi = min(data_dropped['clusters'])
ma = max(data_dropped['clusters'])
cluster_2_id = {} #Dictionary [cluster - ID mapping]
for cluster_no in range(mi, (ma+1)):
    #print((data_dropped.loc[data_dropped['clusters'] == cluster_no])['ID_added'].values)
    cluster_2_id[cluster_no] = list((data_dropped.loc[data_dropped['clusters'] == cluster_no])['ID_added'].values)
    

# Test

In [417]:
#test_data = pd.read_excel('data.xlsx') #This will read new data file with path (test)
test_data = test
test_data['SpecificGravity'] = min_max_scaling(test_data['SpecificGravity'])
test_data = onehot(test_data)
test_data_drop = drop(test_data)
test_clusters = y_km_up.predict(test_data_drop)
test_data['cluster_pred'] = test_clusters
total_samples = len(test_data) #Denominator
match = 0
for index, row in test_data.iterrows():
    cluster = row['cluster_pred']
    person = row['ID_random']
    cluster = int(cluster)
    person = int(person)
    if(person in cluster_2_id[cluster]):
        match = match+1
        
accuracy = match/total_samples*100
print(accuracy)

100.0


/var/folders/jl/4mzp2md15q54v0t2894cffj80000gn/T/ipykernel_87893/2024498184.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['SpecificGravity'] = min_max_scaling(test_data['SpecificGravity'])
